```
This software is a part of GPU Ocean.

Copyright (C) 2020  SINTEF Digital

This notebook sets up a simulation of the complete Norwegian coast and
meassures the time spent in various parts of the code

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

# Assessing computational performance

In [1]:
#Lets have matplotlib "inline"
%matplotlib inline

import os
import sys

#Import packages we need
import numpy as np
from netCDF4 import Dataset
import datetime, time
from IPython.display import display
from IPython.display import clear_output

#For plotting
import matplotlib
from matplotlib import pyplot as plt

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../../')))

from SWESimulators import CDKLM16, Common, IPythonMagic, NetCDFInitialization


In [2]:
%cuda_context_handler gpu_ctx

# Select source url and domain
We assume that the NetCDF file is available locally, since otherwise the initialization will depend a lot on your internet connection

In [3]:
norkyst800_url = 'https://thredds.met.no/thredds/dodsC/fou-hi/norkyst800m-1h/NorKyst-800m_ZDEPTHS_his.an.2019071600.nc'

local_norkyst800_url_havahol = '/media/havahol/Seagate Backup Plus Drive/gpu_ocean/cdklm_practicalities/NorKyst-800m_ZDEPTHS_his.an.2019071600.nc'

source_url = norkyst800_url

if source_url == norkyst800_url and os.path.exists(local_norkyst800_url_havahol):
    source_url = local_norkyst800_url_havahol
print('source url: '+source_url)

source url: /media/havahol/Seagate Backup Plus Drive/gpu_ocean/cdklm_practicalities/NorKyst-800m_ZDEPTHS_his.an.2019071600.nc


In [4]:
casename = 'complete_coast'

folder = casename+"_"+datetime.datetime.now().strftime("%Y_%m_%d")
os.makedirs(folder, exist_ok=True)

# Initialization


In [5]:
initialization_start = time.time()
data_args = NetCDFInitialization.getInitialConditionsNorKystCases(source_url, casename, download_data=False)
sim_args = {
    "gpu_ctx": gpu_ctx,
    "dt": 0.0,
     }

timesteps = data_args["timesteps"][:5]

sim = CDKLM16.CDKLM16(**sim_args, **NetCDFInitialization.removeMetadata(data_args))

initialization_time = time.time() - initialization_start
print('Initialization took '+"{:.2f}".format(initialization_time)+' s')

This will give inaccurate angle along the border!
This will give inaccurate coriolis along the border!


Initialization took 11.79 s


# Running the simulation

In [6]:

def ncSimulation(sim, filename, timesteps, interior_domain_only=False, num_substeps = 3):
    filename = os.path.join(folder, filename)

    simulation_times = None
    writing_times = None
    startup_time = None
    ending_time = None
    
    # Set some common parameters
    netcdf_frequency = 1
    
    
    
    if (os.path.isfile(filename)):
        print("File " + filename + " already exists, skipping simulation.")
        return
    
    try:
        startup_start = time.time()
        ncfile = Dataset(filename, 'w')

        var = {}
        var['eta'], var['hu'], var['hv'] = sim.download(interior_domain_only=interior_domain_only)
        _, var['Hm'] = sim.downloadBathymetry(interior_domain_only=interior_domain_only)

        ny, nx = var['eta'].shape

        # Create dimensions
        ncfile.createDimension('time', None) # unlimited
        ncfile.createDimension('x', nx)
        ncfile.createDimension('y', ny)

        ncvar = {}

        # Create variables for dimensions
        ncvar['time'] = ncfile.createVariable('time', 'f8', ('time',))
        ncvar['x'] = ncfile.createVariable('x', 'f4', ('x',))
        ncvar['y'] = ncfile.createVariable('y', 'f4', ('y',))

        # Fill dimension variables
        ncvar['x'][:] = np.linspace(0, nx*sim.dx, nx)
        ncvar['y'][:] = np.linspace(0, ny*sim.dy, ny)

        # Create static variables
        ncvar['Hm'] = ncfile.createVariable('Hm', 'f8', ('y', 'x',), zlib=True)
        ncvar['Hm'][:,:] = var['Hm'][:,:]

        # Create time varying data variables
        for varname in ['eta', 'hu', 'hv']:
            ncvar[varname] = ncfile.createVariable(varname, 'f8', ('time', 'y', 'x',), zlib=True)
        ncvar['num_iterations'] = ncfile.createVariable('num_iterations', 'i4', ('time',))

        startup_time = time.time() - startup_start
        
        #Simulate n timesteps
        timestep_sizes = timesteps[1:] - timesteps[:-1]
        if netcdf_frequency > 1:
            timestep_sizes = timestep_sizes/netcdf_frequency
            timestep_sizes = np.repeat(timestep_sizes, netcdf_frequency)
        
        timestep_sizes = np.insert(timestep_sizes, 0, 0)
        simulation_times = np.zeros_like(timestep_sizes)
        writing_times    = np.zeros_like(timestep_sizes)
        
        with Common.ProgressPrinter(5) as progress:
            for i, timestep_size in enumerate(timestep_sizes):
                #Don't simulate if first step (store initial conditions also)
                
                simulation_start = time.time()
                if (timestep_size > 0):
                    substep_size = timestep_size / num_substeps
                    for j in range(num_substeps):
                        sim.updateDt()
                        sim.step(substep_size)
                # Store simulation timing
                simulation_times[i] = time.time() - simulation_start
                
                writing_start = time.time()
                var['eta'], var['hu'], var['hv'] = sim.download(interior_domain_only=False)
                ncvar['time'][i] = sim.t
                ncvar['num_iterations'][i] = sim.num_iterations

                abort=False
                for varname in ['eta', 'hu', 'hv']:
                    ncvar[varname][i,:,:] = var[varname][:,:] #np.ma.masked_invalid(var[varname][:,:])
                    if (np.any(np.isnan(var[varname]))):
                        print("Variable " + varname + " contains NaN values!")
                        abort=True

                if (abort):
                    print("Aborting at t=" + str(sim.t))
                    ncfile.sync()
                    break

                # Store writing time
                writing_times[i] = time.time() - writing_start
                
                clear_output(wait = True)
                
                print(progress.getPrintString(i/(len(timestep_sizes)-1)))

    except Exception as e:
        print("Something went wrong:" + str(e))
        raise e
    finally:
        ending_start = time.time()
        ncfile.close()
        ending_time = time.time() - ending_start
        
    return startup_time, simulation_times, writing_times, ending_time

# Original resolution

In [7]:
filename = casename+"_original_"+datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")+".nc"
startup_time, simulation_times, writing_times, ending_time = \
    ncSimulation(sim, filename, timesteps)
sim.cleanUp()

0% [##############################] 100%. Total: 2m 12s, elapsed: 2m 12s, remaining: 0s


In [8]:
print('Initialization: ', initialization_time)
print('startup_time:   ', startup_time)
print('ending_time:    ', ending_time )
print('simulating,    writing')
print(simulation_times)
print(writing_times)


Initialization:  11.792119026184082
startup_time:    0.06632375717163086
ending_time:     0.4920666217803955
simulating,    writing
[2.38418579e-06 3.11285236e+01 3.11171839e+01 3.10936799e+01
 3.10934830e+01]
[0.31024027 2.04635239 2.05815697 2.06335521 2.0562427 ]


# Low resolution

In [21]:
low_res_init_start = time.time()
#Rescaling initial conditions
data_args = NetCDFInitialization.getInitialConditionsNorKystCases(source_url, casename, download_data=False)
data_args_lowres =  NetCDFInitialization.rescaleInitialConditions(data_args, scale = 0.5)


#Setting up simulator, observation, drifters and running as above
sim_lowres = CDKLM16.CDKLM16(**sim_args, **NetCDFInitialization.removeMetadata(data_args_lowres))

low_res_init_time = time.time() - low_res_init_start

This will give inaccurate angle along the border!
This will give inaccurate coriolis along the border!


In [22]:
filename = casename+"_lowres_"+datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")+".nc"
startup_time, simulation_times, writing_times, ending_time = \
    ncSimulation(sim_lowres, filename, timesteps)
sim_lowres.cleanUp()

0% [##############################] 100%. Total: 17s, elapsed: 17s, remaining: 0s


In [23]:
print('Initialization: ', low_res_init_time)
print('startup_time:   ', startup_time)
print('ending_time:    ', ending_time )
print('simulating,    writing')
print(simulation_times)
print(writing_times)


Initialization:  15.647482872009277
startup_time:    0.008220911026000977
ending_time:     0.4609107971191406
simulating,    writing
[2.86102295e-06 3.99219823e+00 3.96007633e+00 3.96133876e+00
 3.96116543e+00]
[0.06466055 0.39173126 0.39138222 0.54767132 0.5456984 ]


# High resolution

In [12]:
high_res_init_start = time.time()
#Rescaling initial conditions
data_args = NetCDFInitialization.getInitialConditionsNorKystCases(source_url, casename, download_data=False)
data_args_highres =  NetCDFInitialization.rescaleInitialConditions(data_args, scale = 2)


#Setting up simulator, observation, drifters and running as above
sim_highres = CDKLM16.CDKLM16(**sim_args, **NetCDFInitialization.removeMetadata(data_args_highres))

high_res_init_time = time.time() - high_res_init_start

This will give inaccurate angle along the border!
This will give inaccurate coriolis along the border!


In [13]:
filename = casename+"_highres_"+datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")+".nc"
startup_time, simulation_times, writing_times, ending_time = \
    ncSimulation(sim_highres, filename, timesteps)
sim_highres.cleanUp()

0% [##############################] 100%. Total: 17m 6s, elapsed: 17m 6s, remaining: 0s


In [14]:
print('Initialization: ', high_res_init_time)
print('startup_time:   ', startup_time)
print('ending_time:    ', ending_time )
print('simulating,    writing')
print(simulation_times)
print(writing_times)


Initialization:  18.275923490524292
startup_time:    0.5963015556335449
ending_time:     0.7340054512023926
simulating,    writing
[3.33786011e-06 2.46630018e+02 2.48069014e+02 2.48639972e+02
 2.48637374e+02]
[1.91580582 8.00296855 8.01734066 8.04286504 8.04189682]


### Retracting computational cost from the original experiments

In [32]:
print('estimate of old per hour cost lowres:   ', ((0*60 + 1)*60+29 - 0 - 0.5*23)/23)
print('estimate of old per hour cost orig res: ', ((0*60 + 10)*60+42 - 0 - 2*23)/23)
print('estimate of old per hour cost highres:  ', ((1*60 + 23)*60+22 - 0 - 8*23)/23)

print('')
print('lowres writing/simulation:   ', 0.5/3.2  , 0.5/3.2  )
print('orig res writing/simulation: ', 2.0/24.8 , (2.0/24.8)*2)
print('highres writing/simulation:  ', 8.0/200.4, (8.0/200.4)*4)

print('')
print('lowres writing/simulation:   ', 0.5/3.4  , 0.5/3.4  )
print('orig res writing/simulation: ', 2.0/25.9 , (2.0/25.9)*2)
print('highres writing/simulation:  ', 8.0/209.5, (8.0/209.5)*4)

estimate of old per hour cost lowres:    3.369565217391304
estimate of old per hour cost orig res:  25.91304347826087
estimate of old per hour cost highres:   209.47826086956522

lowres writing/simulation:    0.15625 0.15625
orig res writing/simulation:  0.08064516129032258 0.16129032258064516
highres writing/simulation:   0.03992015968063872 0.1596806387225549

lowres writing/simulation:    0.14705882352941177 0.14705882352941177
orig res writing/simulation:  0.07722007722007722 0.15444015444015444
highres writing/simulation:   0.03818615751789976 0.15274463007159905


In [34]:
(3.4*8, 3.4*8*8)

(27.2, 217.6)

### Scaling of original experiments

In [43]:
laptop  = [9*60 + 22, 53*60 + 49, (6*60 + 10)*60 +  9]
desktop = [1*60 + 29, 10*60 + 49, (1*60 + 23)*60 + 22]
server  = [       47,  3*60 + 13,         23 *60 + 21]

deskto2 = [(4+0.5)*23, (31.1+2.0)*23, (248.5+8)*23]

print('desktop', desktop)
print('deskto2', deskto2)

print('laptop:  ',  laptop[1]/laptop[0],   laptop[2]/laptop[0],   laptop[2]/laptop[1])
print('desktop: ', desktop[1]/desktop[0], desktop[2]/desktop[0], desktop[2]/desktop[1])
print('server:  ',  server[1]/server[0],   server[2]/server[0],   server[2]/server[1])

print('deskto2: ', deskto2[1]/deskto2[0], deskto2[2]/deskto2[0], deskto2[2]/deskto2[1])


desktop [89, 649, 5002]
deskto2 [103.5, 761.3000000000001, 5899.5]
laptop:   5.745551601423488 39.51779359430605 6.877980799008981
desktop:  7.292134831460674 56.20224719101124 7.707241910631741
server:   4.1063829787234045 29.80851063829787 7.259067357512953
deskto2:  7.355555555555556 57.0 7.749244712990936


In [45]:
np.sqrt(541875)

736.1215932167728